In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

Download the source code for TARELCO II's webpage.

In [6]:
import requests

# URL of the website to scrape
url = "https://www.tarelco2.com/rates.html"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the content of the response (HTML source code)
    html_content = response.text
    
    # Save the HTML source code to a file
    with open("tarelco2_rates.html", "w", encoding="utf-8") as file:
        file.write(html_content)
    
    print("HTML source code has been downloaded and saved as 'tarelco2_rates.html'.")
else:
    print(f"Failed to retrieve the website. Status code: {response.status_code}")

HTML source code has been downloaded and saved as 'tarelco2_rates.html'.


In [8]:
import requests
from bs4 import BeautifulSoup
import os

# Load the HTML file
with open("tarelco2_rates.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# Create a directory to save the PDF files
pdf_dir = "pdf downloads"
if not os.path.exists(pdf_dir):
    os.makedirs(pdf_dir)

# Find all the links in the "Generation Charge" section
for link in soup.find_all('a'):
    href = link.get('href')
    text = link.get_text(strip=True)

    # Check if the link text is a decimal/float value
    try:
        float(text)
        # If it's a float, download the PDF
        pdf_url = f"https://www.tarelco2.com/{href}"
        pdf_response = requests.get(pdf_url)
        
        # Save the PDF file
        pdf_name = os.path.join(pdf_dir, os.path.basename(href))
        with open(pdf_name, "wb") as pdf_file:
            pdf_file.write(pdf_response.content)
        
        print(f"Downloaded: {pdf_name}")
        
    except ValueError:
        # Skip if the text is not a decimal/float
        continue

Downloaded: pdf downloads/GenBreakdown2024-01.pdf
Downloaded: pdf downloads/GenBreakdown2024-02.pdf
Downloaded: pdf downloads/GenBreakdown2024-03.pdf
Downloaded: pdf downloads/GenBreakdown2024-04.pdf
Downloaded: pdf downloads/GenBreakdown2024-05.pdf
Downloaded: pdf downloads/GenBreakdown2024-06.pdf
Downloaded: pdf downloads/GenBreakdown2024-07.pdf
Downloaded: pdf downloads/GenBreakdown2023-01.pdf
Downloaded: pdf downloads/GenBreakdown2023-02.pdf
Downloaded: pdf downloads/GenBreakdown2023-03.pdf
Downloaded: pdf downloads/GenBreakdown2023-04.pdf
Downloaded: pdf downloads/GenBreakdown2023-05.pdf
Downloaded: pdf downloads/GenBreakdown2023-06.pdf
Downloaded: pdf downloads/GenBreakdown2023-07.pdf
Downloaded: pdf downloads/GenBreakdown2023-08.pdf
Downloaded: pdf downloads/GenBreakdown2023-09.pdf
Downloaded: pdf downloads/GenBreakdown2023-10.pdf
Downloaded: pdf downloads/GenBreakdown2023-11.pdf
Downloaded: pdf downloads/GenBreakdown2023-12.pdf
Downloaded: pdf downloads/GenBreakdown2022-01.pdf


### **Processing**

In [13]:
df = tabula.read_pdf("pdf downloads/GenBreakdown2024-01.pdf", lattice=True, pages=1)[0]

In [14]:
df

,SOURCE,%\rTotal kWh\rPurchased,(A)\rKWh Purchased,(B)\rBasic\rGeneration\rCost\r(Php),"©\rOther Cost\rAdjustment (DAA)\rNSS, and Other Billing\radjustment)/ and Discount\r(PhP)",(D = B+ C)\rTotal Generation\rCost for the\rMonth (Php),(D/A)\rAverage\rGeneration\rCost\r(Php/kWH)
0,NPC-TSC,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BILATERAL,NaN,NaN,NaN,NaN,NaN,NaN
3,CONTRACTS W/ IPPs,NaN,NaN,NaN,NaN,NaN,NaN
4,1. GenPower Mariveles_GMEC,29.84%,"10,080,743","58,850,533.51","-653,644.79","58,196,888.73",5.7731
5,1. GenPower Dinginin_GnPD,17.50%,"5,911,000","37,121,493.82","-277,884.00","36,843,609.83",6.2331
6,2. Limay Power Inc.,21.74%,"7,343,777","53,448,028.21","-828,245.29","52,619,782.93",7.1652
7,3. WESM,30.70%,"10,371,263","51,456,407.30",NaN,"51,456,407.30",4.9614
8,SELF-GENERATION,NaN,NaN,NaN,NaN,NaN,NaN
9,SALE FOR RESALE,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df2 = tabula.read_pdf("pdf downloads/GenBreakdown2018-12.pdf", lattice=True, pages=1)[0]
df2

,SOURCE,%\rTotal kWh\rPurchased,(A)\rKWh Purchased,(B)\rBasic\rGeneration\rCostN\r(Php),"©\rOther Cost\rAdjustment (DAA)\rS, and Other Billi\radjustment)\r(PhP)",(D = B+ C)\rTotal Generation\rCost for the\rgMonth (Php),(D/A)\rAverage\rGeneration\rCost\r(Php/kWH)
0,NPC-TSC,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BILATERAL,NaN,NaN,NaN,NaN,NaN,NaN
3,CONTRACTS W/ IPPs,NaN,NaN,NaN,NaN,NaN,NaN
4,1. GenPower Mariveles_GMPC,43.07%,"12,276,000","65,842,763.02","(419,097.72)","65,423,665.30",5.3294
5,2.SMC Cosolidated Power,33.70%,"9,605,784","49,926,268.92","(1,189,722.60)","41,184,599.66",4.2875
6,3.,NaN,NaN,NaN,NaN,NaN,NaN
7,WESM,23.23%,"6,620,766","26,269,480.18",NaN,"26,269,480.18",3.9677
8,SELF-GENERATION,NaN,NaN,NaN,NaN,NaN,NaN
9,SALE FOR RESALE,NaN,NaN,NaN,NaN,NaN,NaN
